In [ ]:
import numpy as np
import pandas as pd

np.random.seed(0)

n_res = 50
n_frames = 20

res_names = [f"A{i}" for i in range(1, n_res + 1)]

df = pd.DataFrame(index=range(n_frames), columns=res_names)

pos = {res: np.random.randn(n_frames, 3) for res in res_names}
ang = {res: np.random.uniform(-180, 180, n_frames) for res in res_names}

path = [1, 16, 17, 30, 36, 50]
for i in range(5):
    curr = f"A{path[i]}"
    nxt = f"A{path[i+1]}"
    pos[nxt] = pos[curr] + 2          # shift all coords by +2
    ang[nxt] = -ang[curr]             # flip angle

# Fill DataFrame (each cell = dict with 4 values)
for f in range(n_frames):
    for res in res_names:
        df.at[f, res] = {
            "x": float(pos[res][f, 0]),
            "y": float(pos[res][f, 1]),
            "z": float(pos[res][f, 2]),
            "angle": float(ang[res][f])
        }

df